# Performance Profiling Examples

This notebook demonstrates various profiling techniques for the MD simulation.

**Topics covered:**
1. Simple Timing with `time.perf_counter()`
2. Function-Level Profiling with `cProfile`
3. Component-Level Timing
4. Memory Profiling with `tracemalloc`
5. IPython Magic Commands (`%timeit`, `%prun`, `%memit`)
6. Visual Profiling with `pyinstrument`

---

## Setup: Import Libraries and Create Simulation

In [ ]:
import sys
import os
import time
import cProfile
import pstats
from io import StringIO

# Add parent directory to path
sys.path.insert(0, os.path.dirname(os.getcwd()))

from md_simulation import TwoParticleMD, Particle, LennardJonesPotential
import numpy as np

print("✅ Libraries imported successfully!")

In [ ]:
# Helper function to create simulation
def create_simulation():
    """Create a standard two-particle simulation."""
    particle1 = Particle(
        position=np.array([5.0, 5.0]), 
        velocity=np.array([0.1, 0.0]), 
        is_fixed=True
    )
    particle2 = Particle(
        position=np.array([8.0, 5.0]), 
        velocity=np.array([-0.05, 0.0])
    )
    potential = LennardJonesPotential()
    sim = TwoParticleMD(particle1, particle2, potential, box_size=(20.0, 20.0))
    return sim

# Test
sim = create_simulation()
print(f"✅ Simulation created: {sim}")

---

## 1. Simple Timing with `time.perf_counter()`

**Best for:** Quick timing of code blocks

**Pros:** Simple, no overhead  
**Cons:** Manual, not detailed

In [ ]:
sim = create_simulation()

# Time single step
start = time.perf_counter()
sim.step()
step_time = time.perf_counter() - start

print(f"Single step time: {step_time*1000:.4f} ms")

# Time 1000 steps
sim = create_simulation()
start = time.perf_counter()
sim.run(1000, record_interval=100)
run_time = time.perf_counter() - start

print(f"1000 steps time: {run_time:.4f} s")
print(f"Average per step: {run_time/1000*1000:.4f} ms")
print(f"Steps per second: {1000/run_time:.1f}")

---

## 2. Function-Level Profiling with `cProfile`

**Best for:** Identifying which functions take the most time

**Pros:** Detailed, built-in  
**Cons:** 10-50% overhead

In [ ]:
sim = create_simulation()

# Profile the simulation
profiler = cProfile.Profile()
profiler.enable()

sim.run(1000, record_interval=100)

profiler.disable()

# Print statistics
s = StringIO()
stats = pstats.Stats(profiler, stream=s)
stats.sort_stats('cumulative')
stats.print_stats(15)  # Top 15 functions

print(s.getvalue())

---

## 3. Component-Level Timing

**Best for:** Understanding which parts of the algorithm are slow

**Approach:** Time each component separately

In [ ]:
sim = create_simulation()
n_iterations = 1000

# Time single step (includes force calculation, integration, boundaries)
start = time.perf_counter()
for _ in range(n_iterations):
    sim.step()
step_time = time.perf_counter() - start

# Time energy calculation
start = time.perf_counter()
for _ in range(n_iterations):
    sim.get_energies()
energy_time = time.perf_counter() - start

# Time force calculation (via potential)
start = time.perf_counter()
for _ in range(n_iterations):
    r_vector = sim.particle1.position - sim.particle2.position
    r = np.linalg.norm(r_vector)
    force_mag = sim.potential.force_magnitude(r)
force_calc_time = time.perf_counter() - start

# Time position update (simplified)
start = time.perf_counter()
for _ in range(n_iterations):
    # Simulate position update
    temp_pos = sim.particle2.position + sim.particle2.velocity * sim.dt
position_update_time = time.perf_counter() - start

# Print results
total_time = step_time + energy_time

print(f"Component timing ({n_iterations} iterations):")
print(f"  Full step (step()):       {step_time:.4f}s ({step_time/total_time*100:.1f}%)")
print(f"  Energy calculation:       {energy_time:.4f}s ({energy_time/total_time*100:.1f}%)")
print(f"  Force calculation only:   {force_calc_time:.4f}s (isolated)")
print(f"  Position update only:     {position_update_time:.4f}s (isolated)")
print(f"  Total:                    {total_time:.4f}s")
print(f"\nAverage time per step: {step_time/n_iterations*1000:.4f} ms")

---

## 4. Memory Profiling with `tracemalloc`

**Best for:** Finding memory leaks and understanding memory usage

**Pros:** Built-in, detailed  
**Cons:** Some overhead

In [ ]:
import tracemalloc

# Start tracing
tracemalloc.start()

# Create and run simulation
sim = create_simulation()
sim.run(1000, record_interval=10)

# Get memory snapshot
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print("Top 10 memory allocations:")
for stat in top_stats[:10]:
    print(f"  {stat}")

# Get current and peak memory
current, peak = tracemalloc.get_traced_memory()
print(f"\nCurrent memory usage: {current / 1024 / 1024:.2f} MB")
print(f"Peak memory usage: {peak / 1024 / 1024:.2f} MB")

tracemalloc.stop()

---

## 5. IPython Magic Commands

**Best for:** Quick interactive profiling in Jupyter

### `%timeit` - Automatic timing with statistics

In [ ]:
# Time a single line
sim = create_simulation()
%timeit sim.step()

In [ ]:
# Time a cell
%%timeit
sim = create_simulation()
sim.run(100)

### `%prun` - Profile with cProfile

In [ ]:
sim = create_simulation()
%prun -s cumulative sim.run(1000)

### `%memit` - Memory usage (requires memory_profiler)

**Note:** Install with `pip install memory_profiler`

In [ ]:
# Uncomment if memory_profiler is installed
# %load_ext memory_profiler
# %memit sim = create_simulation()
# %memit sim.run(1000)

print("To use %memit:")
print("  1. Install: pip install memory_profiler")
print("  2. Load extension: %load_ext memory_profiler")
print("  3. Use: %memit your_code()")

---

## 6. Visual Profiling with `pyinstrument`

**Best for:** Beautiful call stack visualization

**Installation:** `pip install pyinstrument`

In [ ]:
try:
    from pyinstrument import Profiler
    
    profiler = Profiler()
    profiler.start()
    
    # Run simulation
    sim = create_simulation()
    sim.run(1000, record_interval=100)
    
    profiler.stop()
    
    # Print results
    print(profiler.output_text(unicode=True, color=True))
    
    # You can also save HTML report
    # with open('profile.html', 'w') as f:
    #     f.write(profiler.output_html())
    
except ImportError:
    print("❌ pyinstrument not installed")
    print("Install with: pip install pyinstrument")

---

## 7. Scaling Analysis

**Best for:** Understanding how performance scales with problem size

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

step_counts = [100, 500, 1000, 5000, 10000]
times = []

for n_steps in step_counts:
    sim = create_simulation()
    
    start = time.perf_counter()
    sim.run(n_steps, record_interval=max(n_steps//10, 1))
    elapsed = time.perf_counter() - start
    
    times.append(elapsed)

# Create DataFrame
df = pd.DataFrame({
    'Steps': step_counts,
    'Time (s)': times,
    'Time/Step (ms)': [t/n*1000 for t, n in zip(times, step_counts)],
    'Steps/sec': [n/t for n, t in zip(step_counts, times)]
})

print(df.to_string(index=False))

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(step_counts, times, 'o-')
ax1.set_xlabel('Number of Steps')
ax1.set_ylabel('Time (s)')
ax1.set_title('Total Time vs Steps')
ax1.grid(True, alpha=0.3)

ax2.plot(step_counts, df['Time/Step (ms)'], 'o-', color='orange')
ax2.set_xlabel('Number of Steps')
ax2.set_ylabel('Time per Step (ms)')
ax2.set_title('Time per Step vs Steps')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Summary and Recommendations

### Key Findings

Based on the profiling above, you should now know:

1. **Which functions are slowest** (from cProfile)
2. **Which components take most time** (from component timing)
3. **Memory usage patterns** (from tracemalloc)
4. **How performance scales** (from scaling analysis)

### Optimization Strategies

#### 🎯 Quick Wins
- **Vectorize operations** - Use NumPy array operations instead of loops
- **Reduce function calls** - Inline small functions in hot paths
- **Cache results** - Store frequently computed values

#### 🚀 Advanced Optimizations
- **Numba JIT** - Add `@jit` decorator to hot functions (10-100x speedup)
- **Parallel processing** - Use multiprocessing for ensemble runs
- **GPU acceleration** - Offload to GPU for large systems (50-500x speedup)

### Next Steps

1. **Identify bottlenecks** - Focus on functions taking >20% of time
2. **Optimize** - Apply appropriate techniques from above
3. **Re-profile** - Verify improvements with profiling
4. **Iterate** - Repeat until performance is acceptable

### Additional Tools

For more advanced profiling:

```bash
# Generate flame graph with py-spy
py-spy record -o profile.svg -- python md_simulation.py

# Visualize cProfile with SnakeViz
python -m cProfile -o profile.prof md_simulation.py
snakeviz profile.prof

# Line-by-line profiling
kernprof -l -v md_simulation.py
```

### Resources

- **[PROFILING_GUIDE.md](../PROFILING_GUIDE.md)** - Comprehensive profiling guide
- **[PARALLELIZATION_GUIDE.md](../PARALLELIZATION_GUIDE.md)** - Optimization techniques
- **[examples/profile_md.py](profile_md.py)** - Python profiling script

---

*This notebook was created with Augment Agent assistance.*